# Shortest Hops
With everything you've learned, it's time to apply it! You've been hired to build an application that finds the shortest path in a computer network. Use what you've learned to implement this application.
## Graph
The switches and routers will be represented as nodes. The links between each router or switch will be represented as edges.

In [4]:
class Edge(object):
    def __init__(self, node, distance):
        self.node = node
        self.distance = distance


class Node(object):
    def __init__(self, val):
        self.value = val
        self.edges = []

    def add_child(self, node, distance):
        self.edges.append(Edge(node, distance))

    def remove_child(self, del_node):
        if del_node in self.edges:
            self.edges.remove(del_node)

    def __repr__(self):
        return 'Node({})'.format(self.value)


class Graph(object):
    def __init__(self, node_list):
        self.nodes = node_list

    def add_edge(self, node1, node2, distance):
        if node1 in self.nodes and node2 in self.nodes:
            node1.add_child(node2, distance)
            node2.add_child(node1, distance)

    def remove_edge(self, node1, node2):
        if node1 in self.nodes and node2 in self.nodes:
            node1.remove_child(node2)
            node2.remove_child(node1)


node_gateway = Node('gateway')
node_lab = Node('lab')
node_backbone_a = Node('backbone_a')
node_backbone_b = Node('backbone_b')
node_north_building = Node('north_building')
node_east_building = Node('east_building')
node_south_building = Node('south_building')

network_graph = Graph([node_gateway, node_north_building, node_lab, node_east_building, node_south_building, node_backbone_a, node_backbone_b])
network_graph.add_edge(node_gateway, node_lab, 4)
network_graph.add_edge(node_gateway, node_east_building, 6)
network_graph.add_edge(node_gateway, node_north_building, 3)
network_graph.add_edge(node_north_building, node_gateway, 3)
network_graph.add_edge(node_north_building, node_east_building, 4)
network_graph.add_edge(node_lab, node_gateway, 4)
network_graph.add_edge(node_lab, node_south_building, 7)
network_graph.add_edge(node_east_building, node_north_building, 4)
network_graph.add_edge(node_east_building, node_gateway, 6)
network_graph.add_edge(node_east_building, node_south_building, 4)
network_graph.add_edge(node_east_building, node_backbone_a, 5)
network_graph.add_edge(node_south_building, node_lab, 7)
network_graph.add_edge(node_south_building, node_east_building, 4)
network_graph.add_edge(node_south_building, node_backbone_b, 4)
network_graph.add_edge(node_backbone_a, node_east_building, 5)
network_graph.add_edge(node_backbone_a, node_backbone_b, 5)
network_graph.add_edge(node_backbone_b, node_south_building, 4)
network_graph.add_edge(node_backbone_b, node_backbone_a, 5)

## Implementation
Build a lookup table that contains the shortest path between any two points.

In [28]:
import math

def get_lookup_table(network_graph):
    """
    Get a lookup table that contains the shortest path between any two points.

    Parameters
    ----------
    network_graph : Graph
        The computer network graph

    Returns
    -------
    lookup_table : dict of dict
        Lookup table

        Ex.
        If you want to get the shortest path between `node_a` and `node_b`, you would do `lookup_table[node_a][node_b]`.
    """
    
    # NOTE: I have implemented the Floyd-Warsha;; algorithm for all pairs shortedt path as described here: https://www.geeksforgeeks.org/floyd-warshall-algorithm-dp-16/.
    # This algorithm is O(V^3) in time complexity.
    
    num_nodes = len(network_graph.nodes)
    
    # initializing the look-up table to be same as graph except that the disconnected nodes are given a weight of +inf
    lookup_table = { node : {node : float("inf") for node in network_graph.nodes} for node in network_graph.nodes }
    for node in network_graph.nodes:
        lookup_table[node][node] = 0.
        for edge in node.edges:
            lookup_table[node][edge.node] = edge.distance

    for intermediate in network_graph.nodes:
        for source in network_graph.nodes:
            for target in network_graph.nodes:
                if lookup_table[source][target] > lookup_table[source][intermediate]+lookup_table[intermediate][target]:
                    lookup_table[source][target] = lookup_table[source][intermediate]+lookup_table[intermediate][target]
    return lookup_table

Once the implementation is done, you can test it below.

In [29]:
lookup_table = get_lookup_table(network_graph)

print('Shortest Path from {} to {} is {}'.format(
    node_gateway.value, node_south_building.value, lookup_table[node_gateway][node_south_building]))

Shortest Path from gateway to south_building is 10


## Solution
If you're having trouble solving the problem, you can find the solution [here](shortest_hops_solution.ipynb).

In [6]:
from collections import defaultdict, deque
import heapq

def djikstra(begin, N):
    costs = [float('inf')]*(N + 1)
    seen = set()
    costs[begin] = 0
    q = []
    heapq.heappush(q, (0,  (0, begin, 0, 0)))
    #path = []
    hops = 0
    final = float('inf')
    node = -1
    while q:
        (cost, (start, node, prev_weight, hops)) = heapq.heappop(q)
        if node == N and cost == costs[N]:
            final = min(final, hops)
        if (start, node, prev_weight, hops) in seen:
            continue
        seen.add((start, node, prev_weight, hops))
        for (next_node, weight) in graph[node]:
            if costs[node] + weight <= costs[next_node]:
                costs[next_node] = costs[node] + weight
                if prev_weight != 0:
                    heapq.heappush(q, (cost + weight, (node, next_node, weight, hops + (weight % 2 != prev_weight % 2) )))
                else:
                    heapq.heappush(q, (cost + weight, (node, next_node, weight, hops)))
    return costs, final



In [3]:
"""
## Process inputs

N, M = map(int, input().strip().split(' '))
graph = defaultdict(set)
edges = []
for _ in range(M):
    x, y, c = map(int, input().strip().split(' '))
    #edges.append((c, x, y))
    #edges.append((c, y, x))
    graph[x].add((y, c))
    graph[y].add((x, c))

weights, hops = djikstra(1, N)
print("%d %d" % (weights[-1], hops))

"""

'\n## Process inputs\n\nN, M = map(int, input().strip().split(\' \'))\ngraph = defaultdict(set)\nedges = []\nfor _ in range(M):\n    x, y, c = map(int, input().strip().split(\' \'))\n    #edges.append((c, x, y))\n    #edges.append((c, y, x))\n    graph[x].add((y, c))\n    graph[y].add((x, c))\n\nweights, hops = djikstra(1, N)\nprint("%d %d" % (weights[-1], hops))\n\n'